# Text Generation using LSTMs
### ディープラーニングによる文章生成

![title](https://camo.githubusercontent.com/9a5b885799c2d8e50f3f049fde2ada7696e974ca/68747470733a2f2f692e696d6775722e636f6d2f484646575674432e706e673f32)

## What is LSTM?
  
テキスト生成は一種の言語モデリング問題。  
言語モデリングは、テキスト読み上げ、会話システム、テキスト要約など、多数の自然言語処理タスクの中心的な問題。    
訓練された言語モデルは、テキストで使用されている前の一連の単語に基づいて、単語の出現の可能性を学習する。  
言語モデルは、文字レベル、n-gramレベル、文レベル、または段落レベルでも操作できる。  
このノートでは、最先端のリカレントニューラルネットワークを実装してトレーニングすることによって、自然言語テキストを生成するための言語モデルを作成する方法について説明していく
。

## 今回はニュースの本文からタイトルを自動生成します  
  
#### Process  
1. データの準備  
2. 文章のお掃除（記号削除、小文字統一）  
3. 単語に切り分ける  
4. トークン化＝数値化  
5. パディングで変数の長さを統一  
6. LSTMの実装

## 1. Import the libraries ライブラリのインポート

In [63]:
import pandas as pd
import numpy as np

## 2. Load the dataset データのロード

In [64]:
import os
path = os.getcwd()
print(path)

/Users/akr712/Desktop/Portfolio/Natural Language Processing/Text_Generation_using_LSTMs


In [65]:
headlines = []
for filename in os.listdir(path):
    if "Articles" in filename:
        article_df = pd.read_csv(path + "/" + filename)
        headlines.extend(list(article_df["headline"].values))
        break
        
headlines = [ h for h in headlines if h != "Unknown" ]
print("The number of headline is :", len(headlines))

The number of headline is : 829


In [66]:
headlines

['N.F.L. vs. Politics Has Been Battle All Season Long',
 'Voice. Vice. Veracity.',
 'A Stand-Up’s Downward Slide',
 'New York Today: A Groundhog Has Her Day',
 'A Swimmer’s Communion With the Ocean',
 'Trail Activity',
 'Super Bowl',
 'Trump’s Mexican Shakedown',
 'Pence’s Presidential Pet',
 'Fruit of a Poison Tree',
 'The Peculiar Populism of Donald Trump',
 'Questions for: ‘On Alaska’s Coldest Days, a Village Draws Close for Warmth’',
 'The New Kids',
 'What My Chinese Mother Made',
 'Do You Think Teenagers Can Make a Difference in the World?',
 'President Pledges to Let Politics Return to Pulpits',
 'The Police Killed My Unarmed Son in 2012. I’m Still Waiting for Justice.',
 'Video of Sheep Slaughtering Ignites a Dispute',
 'This Will Change Your Mind',
 'Busy Start for a President, and That Was in 1933',
 'Trump Reverts to Pillars of Obama Policies Abroad',
 'Should Dollar Rise or Fall? The Trump Team’s Message Is Garbled',
 'Hardware for the Modern Winemaker',
 'Cajun, Far From H

## 3. Dataset preparation 前処理

### 3.1 Dataset cleaning  データクリーニング
  
記号を取り除き、「文字」と「数字」だけ残す。また小文字で統一する。

In [67]:
import string
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

def clean_text(headline):
    text = "".join( word for word in headline if word not in string.punctuation ).lower()
    text = text.encode("utf8").decode("ascii", "ignore")
    return text

# 元データのタイトルに含まれる単語群から独自のコーパスを作成
corpus = [ clean_text(headline) for headline in headlines ]
corpus[:5]

['nfl vs politics has been battle all season long',
 'voice vice veracity',
 'a standups downward slide',
 'new york today a groundhog has her day',
 'a swimmers communion with the ocean']

### 3.2 Generating Sequence of N-gram Tokens 文章の単語化&数値化

![title](https://static.magento.com/sites/default/files/images/Tokenization_Blogpost_body_644x362.jpg)

自然言語処理では、テキストを単語単位に分解してベクトル化するのが主流である。  
N-gram は、Morphological Analysis（形態要素解析）に並ぶ代表的な単語の切り出し手法のひとつ。  
具体的には、N-gramとは自然言語（テキスト）を連続するN個の文字、もしくはN個の単語単位で単語を切り出す手法のこと。  
強みは「コーパス」が事前に入らないこと、弱みは切り出した単語数が肥大化しやすい点。  
ex:) "I voted for Trump." n=2 => "I voted", "for Trump"

In [69]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## トークン化
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    input_seqences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_seqences.append(n_gram_sequence)
            
    return input_seqences, total_words

input_sequences, total_words = get_sequence_of_tokens(corpus)
input_sequences[:10]

[[660, 117],
 [660, 117, 72],
 [660, 117, 72, 73],
 [660, 117, 72, 73, 661],
 [660, 117, 72, 73, 661, 662],
 [660, 117, 72, 73, 661, 662, 63],
 [660, 117, 72, 73, 661, 662, 63, 29],
 [660, 117, 72, 73, 661, 662, 63, 29, 210],
 [211, 663],
 [211, 663, 664]]


Headline: i stand with the shedevils  
Ngrams: | Sequence of Tokens  
  
Ngram	Sequence of Tokens  
i stand	[30, 507]  
i stand with	[30, 507, 11]  
i stand with the	[30, 507, 11, 1]  
i stand with the shedevils	[30, 507, 11, 1, 975]  
  

### 3.3 Padding the Sequences and obtain Variables
#### パディングによって固定長データを作り、説明変数を得る

Paddingとは？  
データを固定長として扱いたいときに、短いデータの前や後に無意味なデータを追加して長さを合わせる処理のことをパディングという。  
ここではトークン化された単語の列の長さは、タイトルごとによって異なる。パディングはこの異なる長さを統一するために行われる。

In [73]:
from keras.preprocessing.sequence import pad_sequences
import keras.utils as ku

def generate_padded_sequences(input_sequences):
    """トークン化されたタイトルの長さが最長のものに長さに統一する"""
    max_sequence_len = max([ len(x) for x in input_sequences ])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding="pre"))
    
    """説明変数と目的変数に分割する"""
    predictors, label = input_sequences[:, -1], input_sequences[:, -1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(input_sequences)

## 4. LSTMs for Text Generation 長短期記憶層アルゴリズムの実装

![title](https://cdn-images-1.medium.com/max/1600/1*yBXV9o5q7L_CvY7quJt3WQ.png)

### 4.1 LSTM ( Long Short-Term Memory  )    
  
1. Input Layer : Takes the sequence of words as input  
2. LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later.  
3. Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer.  
4. Output Layer : Computes the probability of the best possible next word as output  

In [71]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

def lstm_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    """入力層"""
    model.add(Embedding(input_dim=total_words, output_dim=10, input_length=input_len))
    
    """隠れ層"""
    model.add(LSTM(units=100))
    model.add(Dropout(rate=0.1))
    
    """出力層 活性化関数は多層のソフトマックス関数"""
    model.add(Dense(units=total_words, activation="softmax"))
    
    """損失関数と最適化手法の設定"""
    model.compile(loss="categorical_crossentropy", optimizer="adam")
    
    return model

model1 = lstm_model(max_sequence_len, total_words)
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 16, 10)            22880     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2288)              231088    
Total params: 298,368
Trainable params: 298,368
Non-trainable params: 0
_________________________________________________________________


#### LSTMモデルにテストデータを学習させていく！

In [ ]:
"""予測ラベルと正解ラベルを用意する"""
model1.fit(predictors, label, epochs=100, verbose=5)

### 4.2 GRU ( Gated recurrent unit )

In [49]:
from keras.layers import GRU

def gru_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    """入力層"""
    model.add(Embedding(input_dim=total_words, output_dim=10, input_length=input_len))
    
    """隠れ層"""
    model.add(GRU(units=100))
    model.add(Dropout(rate=0.1))
    
    """出力層 活性化関数は多層のソフトマックス関数"""
    model.add(Dense(units=total_words, activation="softmax"))
    
    """損失関数と最適化手法の設定"""
    model.compile(loss="categorical_crossentropy", optimizer="adam")
    
    return model

model2 = gru_model(max_sequence_len, total_words)
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 16, 10)            22880     
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               33300     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2288)              231088    
Total params: 287,268
Trainable params: 287,268
Non-trainable params: 0
_________________________________________________________________


#### GRUモデルにテストデータを学習させていく！

In [ ]:
"""予測ラベルと正解ラベルを用意する"""
model2.fit(predictors, label, epochs=100, verbose=5)

## 5. Generating the text タイトルにふさわしいテキストを自動生成する

1.  tokenize the seed text  
2. pad the sequences  
3. pass into the trained model to get predicted word

In [76]:
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

def generate_text(seed_text, next_words, model, max_sequence):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequence([seed_test])[0]
        token_list = pad_sequences(sequences=[token_list], maxlen=max_sequence_len -1, padding="pre")
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
                
            seed_text += " " +output_word
        return seed_text.title